In [ ]:
datadir = ""
inpath = "/dls/science/groups/das/ExampleData/NeXus/newnexuswriter2024/i16/i16_nexus_test_04Feb25/1078060.nxs"
outpath = "/dls/science/groups/das/ExampleData/NeXus/newnexuswriter2024/i16/i16_nexus_test_04Feb25/1078060_remap_test.nxs"

# Run MSMapper and Analyse Results

In [ ]:
import sys, os
import h5py
import numpy as np
import matplotlib.pyplot as plt

import hdfmap
from i16_msmapper import mapper_runner

print("sys.path: ", sys.path)
print(hdfmap.module_info())

# set the location of msmapper
CONFIG = '/tmp/msmapper_config'
MSMAPPER = '/opt/msmapper/msmapper'
mapper_runner.SHELL_CMD = ' '.join([MSMAPPER, "-configuration", CONFIG, "-bean %s"])

if not os.path.exists(MSMAPPER):
    raise ValueError("MSMAPPER not found")

# make config dir
os.makedirs(CONFIG, exist_ok=True)

if not outpath:
    outpath = '/tmp/msmapper_remap.nxs'

## Run MSMapper

In [ ]:
mapper_runner.run_msmapper(
    input_files=[inpath, ''],
    output_file=outpath
)
print(f"File created {outpath}: {os.path.isfile(outpath)}")

## Load MSMapper output

In [ ]:
import h5py
filename = os.path.basename(outpath)

with h5py.File(outpath, 'r') as hdf:
    # the following are links to the original scan file
    scan_command = hdf['/entry0/scan_command'].asstr()[()]  # str
    crystal = hdf['/entry0/sample/name'].asstr()[()]  # str
    temp = hdf.get('/entry0/instrument/temperature_controller/Tsample', np.array(0))[()]  # float
    unit_cell = np.reshape(hdf['/entry0/sample/unit_cell'], -1)  # 1D array, length 6
    energy = hdf['/entry0/sample/beam/incident_energy'][()]  # # float
    ubmatrix = hdf['/entry0/sample/ub_matrix'][0]  # 3D array, shape (3,3)
    # this is the processed data
    haxis = hdf['/processed/reciprocal_space/h-axis'][()]  # 1D array, length n
    kaxis = hdf['/processed/reciprocal_space/k-axis'][()]  # 1D array, length m
    laxis = hdf['/processed/reciprocal_space/l-axis'][()]  # 1D array, length o
    volume = hdf['/processed/reciprocal_space/volume'][()]  # 3D array, shape [n,m,o]
    # detector parameters
    pixel_size = hdf['/entry0/instrument/pil3_100k/module/fast_pixel_direction'][()] # float, mm
    detector_distance = hdf['/entry0/instrument/pil3_100k/transformations/origin_offset'][()] # float, mm

print(f"Loaded file: {filename} with volume shape: {volume.shape}")

In [ ]:
# average angle subtended by each pixel
solid_angle = pixel_size ** 2 / detector_distance ** 2  # sr
if solid_angle.size > 0:
    solid_angle = solid_angle[0]
print(f'Each pixel is normalised by the solid angle: {solid_angle: .4g} sr')

volume = volume * solid_angle

In [ ]:
# Plot summed cuts
plt.figure(figsize=[18, 8], dpi=60)
title = f"{filename} '{crystal}' {temp:.3g} K\n{scan_command}"
plt.suptitle(title, fontsize=18)

plt.subplot(131)
plt.plot(haxis, volume.sum(axis=1).sum(axis=1))
plt.xlabel('h-axis (r.l.u.)', fontsize=16)
plt.ylabel('sum axes [1,2]', fontsize=16)

plt.subplot(132)
plt.plot(kaxis, volume.sum(axis=0).sum(axis=1))
plt.xlabel('k-axis (r.l.u.)', fontsize=16)
plt.ylabel('sum axes [0,2]', fontsize=16)

plt.subplot(133)
plt.plot(laxis, volume.sum(axis=0).sum(axis=0))
plt.xlabel('l-axis (r.l.u.)', fontsize=16)
plt.ylabel('sum axes [0,1]', fontsize=16)

# Plot summed images
plt.figure(figsize=[18, 8], dpi=60)
title = f"{filename}\n{crystal} {temp:.3g} K: {scan_command}"
plt.suptitle(title, fontsize=20)
plt.subplots_adjust(wspace=0.3)

plt.subplot(131)
K, H = np.meshgrid(kaxis, haxis)
plt.pcolormesh(H, K, volume.sum(axis=2), shading='auto')
plt.xlabel('h-axis (r.l.u.)', fontsize=16)
plt.ylabel('k-axis (r.l.u.)', fontsize=16)
plt.axis('image')
#plt.colorbar()

plt.subplot(132)
L, H = np.meshgrid(laxis, haxis)
plt.pcolormesh(H, L, volume.sum(axis=1), shading='auto')
plt.xlabel('h-axis (r.l.u.)', fontsize=16)
plt.ylabel('l-axis (r.l.u.)', fontsize=16)
plt.axis('image')
#plt.colorbar()

plt.subplot(133)
L, K = np.meshgrid(laxis, kaxis)
plt.pcolormesh(K, L, volume.sum(axis=0), shading='auto')
plt.xlabel('k-axis (r.l.u.)', fontsize=16)
plt.ylabel('l-axis (r.l.u.)', fontsize=16)
plt.axis('image')
plt.colorbar()

plt.show()

## NXtransformations
plot reciprocal space and instrument positions from NXtransformations

In [ ]:

from i16_msmapper.nx_transformations import NXScan

with h5py.File(inpath) as nxs:
    scan = NXScan(nxs)
    scan.plot_wavevectors()

    scan.plot_instrument()

    plt.show()